In [55]:
import pandas as pd
import os

In [56]:
path = os.path.join('OriginalData', 'skills.csv')
data = pd.read_csv(path)
data_backup = data.copy()

In [57]:
# Show amount of unique skills
data['skill'].nunique()

53025

In [58]:
# Lowercase all the skills
data['skill'] = data['skill'].str.lower()

In [59]:
# remove trailing and leading spaces
data['skill'] = data['skill'].str.strip()

In [60]:
# Remove skills that are longer than 3 words
data['skill'] = data['skill'].str.split().apply(lambda x: ' '.join(x[:3]))

In [61]:
# Drop duplicates
data = data.drop_duplicates()

In [62]:
# Group data by user_id
data = data.groupby('user_id')['skill'].apply(list).reset_index()
data

,user_id,skill
0,1,"[mühendislik, eğitim]"
1,2,"[android, java, 3d studio max, müşteri hizmetl..."
2,3,"[mühendislik, programlama, algoritma tasarımı]"
3,5,"[oyun tasarımı, veritabanı yönetimi, veri bili..."
4,6,"[liderlik, ekip liderliği, mühendislik, eğitim..."
...,...,...
62397,66269,"[cloud computing, strategic partnerships, ente..."
62398,66270,"[pmp, stratejik yönetim, stratejik bilişim yön..."
62399,66271,"[software development, asp.net, c#, programmin..."
62400,66272,"[java, tomcat, mysql, software development, so..."


In [63]:
# Create a new column with the amount of skills per user
data['skill_count'] = data['skill'].apply(lambda x: len(x[0]))
data

,user_id,skill,skill_count
0,1,"[mühendislik, eğitim]",11
1,2,"[android, java, 3d studio max, müşteri hizmetl...",7
2,3,"[mühendislik, programlama, algoritma tasarımı]",11
3,5,"[oyun tasarımı, veritabanı yönetimi, veri bili...",13
4,6,"[liderlik, ekip liderliği, mühendislik, eğitim...",8
...,...,...,...
62397,66269,"[cloud computing, strategic partnerships, ente...",15
62398,66270,"[pmp, stratejik yönetim, stratejik bilişim yön...",3
62399,66271,"[software development, asp.net, c#, programmin...",20
62400,66272,"[java, tomcat, mysql, software development, so...",4


In [64]:
# Drop skill
data = data.drop(columns=['skill'])
data

,user_id,skill_count
0,1,11
1,2,7
2,3,11
3,5,13
4,6,8
...,...,...
62397,66269,15
62398,66270,3
62399,66271,20
62400,66272,4


In [65]:
# Save data to a new csv file in PreparedData folder
data.to_csv(os.path.join('PreparedData', 'skills.csv'), index=False)